# Introduction

Here, I am going to crop leaf image using Canny edge detection on Plant pathology competion images.<br>
I hope croping leaf from original image and, train model on it will increase overall metric scores.<br>
Bellow is the notebook that I refered to. You should check this since it's very helpful and interesting !!<br>
https://www.kaggle.com/tarunpaparaju/plant-pathology-2020-eda-models#Image-processing-and-augmentation-
<br>

Contents of this notebook is as follows
1. What is Canny edge detection
2. Understanding images in Plant pathology competion
3. How to crop leaf from original image using Canny detection

# What is Canny edge detection?
 Canny Edge Detection is a popular edge detection algorithm. It is often used in image processing.<br>
 We can crop the area of leaf for image in plant pathology competion, since images contains only single leaf and background.<br>
 See bellow for detail in algorithm of Canny Edge detection.<br>
 https://docs.opencv.org/master/da/d22/tutorial_py_canny.html

# Understanding images in Plant pathology competion
First we load dataPILnd see the distribution of labels and show a few images.<br>
After preform statistical anaysis on dataset.

## Imports

In [ ]:
import os
import glob
import math
import json

import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2

## Load data

In [ ]:
BASE_DIR = "../input/plant-pathology-2021-fgvc8"

In [ ]:
# Training data image path and label
train_image_path = glob.glob(os.path.join(BASE_DIR, "train_images/*.jpg"))
label_df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

# Diplay number of images and label dataframe
print("Number of training images: {}".format(len(train_image_path)))
display(label_df.head(10))

Long label name is included such as "scab frog_eye_leaf_spot complex".<br>
Image that has multible labels. We are going to see those images later

## Label distribution

In [ ]:
# Count number of labels
label_count = pd.DataFrame(label_df.labels.value_counts())
label_count["proportion"] = label_count / len(label_df) * 100
display(label_count)

In [ ]:
# Show distribution for label by bar graph and pie chart 
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
label_count["labels"].plot.bar(ax=ax[0], title="Number of data per label", rot=90, fontsize=15)
label_count["proportion"].plot.pie(ax=ax[1], title="Ratio of label", autopct='%1.1f%%', fontsize=15)
plt.show()

From above label distribution we could see bellow <br>
1. Main categories are bellow<br>
    - Healthy
    - Scab
    - Frog_eye_leaf_spot
    - Rust
    - Powdery_mildew
    - Complex
    - Multiple label of above
2. Mixed labels consists of 20% of total

## Show few images
We have seen label distribution of images at former section. Here I am showing a few images for each label.

In [ ]:
display_image_num = 9

for label in label_count.index:
    df_tmp = label_df[label_df.labels == label]
    image_file_names = df_tmp.sample(display_image_num).image.values
    images = [os.path.join(BASE_DIR, "train_images/", file_name) for file_name in image_file_names]
    
    # Show 3X3 images per each label
    plt.figure(figsize=(15, 15))
    for idx, image in enumerate(images):
        plt.subplot(math.ceil(display_image_num / 3), 3, idx + 1)
        im = plt.imread(image)
        plt.imshow(im)
    print("\n==================== Label:{} ===================".format(label))
    plt.tight_layout()
    plt.show()

From images we can see bellow
- “complex” is leaves with too many diseases to classify (Mentioned in official discription)
- Labels "frog_eye_leaf_spot", "rust", "powdery_mildew" seems easy to identify
- “scab” seems difficult to identify spots<br>
    -> Consider applying filter to emphasize scab

## Statistical analysis on image data
Here we statisticaly analyze image data to get insight on image pattern.

## Image size distribution
Image size is important because we need to resize all the image to same size in order to train model.

In [ ]:
# Get image size
image_property = {"image": [], "width": [], "height": []}
for image_path in train_image_path:
    im = Image.open(image_path)
    width, height = im.size
    file_name = os.path.basename(image_path)
    image_property["image"].append(file_name)
    image_property["width"].append(width)
    image_property["height"].append(height)

In [ ]:
# Merge width, height info with label data
image_size = pd.DataFrame(image_property)
image_size = pd.merge(image_size, label_df, on="image", how="outer")
display(image_size.head())
display(image_size.info())

In [ ]:
# Plot (width, height) pair
image_size.plot(x="width", y="height", linestyle="none", marker = "x")

In [ ]:
# Count value pair for (width, height)
image_size.groupby(["width", "height"]).count()

We can see
- All the images do not have the same size.
- There are 9 types of images size
- 4000X2672 is the most popular size

### Analysis on single image
Let's take a look at single image and see the pixel value distribution to get an insight.

In [ ]:
# Sampling image to analayse
idx = 2399 # Randomly selected index
img_path, label = label_df.iloc[idx].values
img_path = os.path.join(BASE_DIR, "train_images/", img_path)
img_sample = plt.imread(img_path)
print("Array shape: {}".format(img_sample.shape))
plt.imshow(img_sample)
plt.title(label)
plt.show()

In [ ]:
# Extract RGB channel
red_channel   = img_sample[:,:,0].flatten()
green_channel = img_sample[:,:,1].flatten()
blue_channel  = img_sample[:,:,2].flatten()
print("red_channel shape: {}".format(red_channel.shape))
print("green_channel shape: {}".format(green_channel.shape))
print("blue_channel shape: {}".format(blue_channel.shape))

In [ ]:
# Show distribution for each channel
y_min, y_max = 0., 0.018
plt.figure(figsize=(20,4))
plt.subplot(1,3,1)
plt.hist(red_channel,bins=50, color="red", alpha=0.3, density=True)
plt.title("Red channel distribution")
plt.ylim([y_min, y_max])

plt.subplot(1,3,2)
plt.hist(green_channel,bins=50, color="green", alpha=0.3, density=True)
plt.title("Green channel distribution")
plt.ylim([y_min, y_max])

plt.subplot(1,3,3)
plt.hist(blue_channel,bins=50, color="blue", alpha=0.3, density=True)
plt.title("Blue channel distribution")
plt.ylim([y_min, y_max])

plt.show()

In [ ]:
# View all channel together
plt.hist(red_channel,bins=50, color="red", alpha=0.3, density=True)
plt.hist(green_channel,bins=50, color="green", alpha=0.3, density=True)
plt.hist(blue_channel,bins=50, color="blue", alpha=0.3, density=True)
plt.ylim([y_min, y_max])
plt.show()

From the distribution from above we can see<br>
1. Green channel has larger density overall
2. Green channel is dense around value 250
3. Red and Blue channel has similar distribution
<br>

The fact that green channel has high density is consistent to tha fact that, images we are handling is leaf therefore contains a lot of green pixles.

### Analysis on entire image
Here we analyze pixel values dependencies for each label.<br>
Since there are only a few images with multiple labels, we only analyze images with labels bellow<br>
"scab", "healthy", "frog_eye_leaf_spot", "rust", "complex", "powdery_mildew"<br>

Steps to analyze are as follows
1. Extract images with certain label
2. Calculate mean value of red, green, blue channel for each image
3. Plot RGB mean values distribution for certain label images
4. Repeat 1. to 3. for all 6 labels

In [ ]:
lables_to_analyze = ["scab", "healthy", "frog_eye_leaf_spot", "rust", "complex", "powdery_mildew"]

In [ ]:
results = dict((key, None) for key in lables_to_analyze)

for label in lables_to_analyze:
    # File name with certain label
    df_tmp = label_df[label_df["labels"]==label]
    image_file_names = df_tmp.image.values
    images = [os.path.join(BASE_DIR, "train_images/", file_name) for file_name in image_file_names]
    
    rc_mean, gc_mean, bc_mean = [], [], []
    for image in images:
        im = plt.imread(image)
        rc_mean.append(im[:,:,0].flatten().mean())
        gc_mean.append(im[:,:,1].flatten().mean())
        bc_mean.append(im[:,:,2].flatten().mean())
    results[label] = {
        "r": rc_mean,
        "g": gc_mean,
        "b": bc_mean
    }

In [ ]:
# Convert calculated result to DataFrame
labels = list(results.keys())
df_list = {}

for label in labels:
    data_array = np.array([
        results[label]["r"],
        results[label]["g"],
        results[label]["b"]]).T
    df_list[label] = pd.DataFrame(
        data_array, columns=["r", "g", "b"])

In [ ]:
# Plot mean RGB distribution for each label
plt.figure(figsize=(8, 20))
gs = gridspec.GridSpec(len(labels), 1)
for idx, label in enumerate(labels):
    ax = plt.subplot(gs[idx])
    
    df_plot = df_list[label]
    sns.distplot(df_plot["r"],bins=10, color='crimson')
    sns.distplot(df_plot["b"],bins=10, color='royalblue')
    sns.distplot(df_plot["g"],bins=10, color='green')
    ax.set_xlabel(None)
    ax.set_title(label, loc='left')
    ax.legend(["Red", "Blue", "Green"])
    ax.set_xlim([0, 255])
    ax.set_ylim([0, 0.05]) 

From the graphs above we can see that "rust" images have realtively flat distribution.<br>
Also, "frog_eye_leaf_spot", "complex", "powdery_mildew" has sharp distribution.<br>
We categorized the shape of distribution bellow<br>
- Sharp distribution:<br>
    "frog_eye_leaf_spot", "complex", "powdery_mildew"
- Intermediate distribution: <br>
    "scab", "health"
- Wide distribution:　<br>
    "rust"
    
Images with simillar distribution may be difficult to identify each other.

# Crop leaf from original image using Canny detection

## Apply canny edge detection on single image

First of all I will show hot to perform canny edge detection on single image.<br>
Also, I will crop leaf part from the entire image.

In [ ]:
# Extract image for edge detecting sample. (No meaning in index number that is hard coded.)
sample_index = 2344
file_name = label_df.iloc[sample_index]["image"]
sample_img_path = os.path.join(BASE_DIR, "train_images/", file_name)
sample_img = plt.imread(sample_img_path)

plt.imshow(sample_img)

In [ ]:
# Apply Cany edge detection
sample_img_edge1 = cv2.Canny(sample_img, 5, 200)
print(sample_img_edge1.shape)
plt.imshow(sample_img_edge1, cmap="gray")

Spot shown in figure above is the pixel where edge is present.<br>
We crop leaf image by finding min and max coordinate in X and Y axis.

In [ ]:
# Crop leaf image from edge detected image
# Extracting boundary box
y_idxs, x_idxs = np.where(sample_img_edge1 > 0)
x_min, x_max = x_idxs.min(), x_idxs.max()
y_min, y_max = y_idxs.min(), y_idxs.max()

# Add boundary to original image
img_with_boundary = sample_img.copy()
line_size = 10

# Vertical boundary (Left)
img_with_boundary[y_min:y_max, x_min-line_size:x_min+line_size, 0] = 255
# Vertical boundary (Right)
img_with_boundary[y_min:y_max, x_max-line_size:x_max+line_size, 0] = 255
# Horizontal boundary (Lower)
img_with_boundary[y_min-line_size:y_min+line_size, x_min:x_max, 0] = 255
# Vertical boundary (Upper)
img_with_boundary[y_max-line_size:y_max+line_size, x_min:x_max, 0] = 255

plt.figure(figsize=(15, 4))
plt.subplot(121)
plt.imshow(img_with_boundary)
plt.title("Leaf image with bounding box")
plt.subplot(122)
plt.imshow(img_with_boundary[y_min:y_max, x_min:x_max])
plt.title("Croped leaf image")

Not a good croping though the leaf is foucused.<br>

Cany edge detection requires min and max threshold for detecting.<br>
We need to find best values since this affects detection result.<br>
Let's start with defining function to crop image.

In [ ]:
def get_boundary(img, min_thresh, max_thresh):
    edge_image = cv2.Canny(img, min_thresh, max_thresh)
    y_idxs, x_idxs = np.where(edge_image > 0)
    x_min, x_max = x_idxs.min(), x_idxs.max()
    y_min, y_max = y_idxs.min(), y_idxs.max()
    return x_min, x_max, y_min, y_max

def get_image_with_boundary(img, x_min, x_max, y_min, y_max):
    img_with_boundary = img.copy()
    line_size = 10
    
    # Vertical boundary (Left)
    img_with_boundary[y_min:y_max, x_min-line_size:x_min+line_size, 0] = 255
    # Vertical boundary (Right)
    img_with_boundary[y_min:y_max, x_max-line_size:x_max+line_size, 0] = 255
    # Horizontal boundary (Lower)
    img_with_boundary[y_min-line_size:y_min+line_size, x_min:x_max, 0] = 255
    # Vertical boundary (Upper)
    img_with_boundary[y_max-line_size:y_max+line_size, x_min:x_max, 0] = 255
    
    return img_with_boundary

def get_and_plot_image_with_boundary(img, min_thresh, max_thresh):
    x_min, x_max, y_min, y_max = get_boundary(img, min_thresh, max_thresh)
    img_with_boundary = get_image_with_boundary(img, x_min, x_max, y_min, y_max)
    plt.imshow(img_with_boundary)

In [ ]:
# Perform edge detection with various threshold
plt.figure(figsize=(15,6))

# minThresh=5, maxThresh=200
plt.subplot(2,3,1)
get_and_plot_image_with_boundary(sample_img, 5, 200)
plt.title("minThresh=5, maxThresh=200")

# minThresh=50, maxThresh=200
plt.subplot(2,3,2)
get_and_plot_image_with_boundary(sample_img, 50, 200)
plt.title("minThresh=50, maxThresh=200")

# minThresh=100, maxThresh=200
plt.subplot(2,3,3)
get_and_plot_image_with_boundary(sample_img, 100, 200)
plt.title("minThresh=100, maxThresh=200")

# minThresh=5, maxThresh=100
plt.subplot(2,3,4)
get_and_plot_image_with_boundary(sample_img, 5, 100)
plt.title("minThresh=5, maxThresh=100")

# minThresh=30, maxThresh=100
plt.subplot(2,3,5)
get_and_plot_image_with_boundary(sample_img, 30, 100)
plt.title("minThresh=30, maxThresh=100")

# minThresh=50, maxThresh=100
plt.subplot(2,3,6)
get_and_plot_image_with_boundary(sample_img, 50, 100)
plt.title("minThresh=50, maxThresh=100")

plt.tight_layout()

With threshold range is bellow 100 (images at 2nd row), entire image has been croped which is meaningless.<br>
With threshold range is bellow 200 (images at 1st row), leaf part is croped but no big differnece with each other.<br>
Next we are going to see how minThresh, maxThresh affects cropping.

## Crop leaf from image

Now that we sawa how to perform Canny edge detection and crop leaf on a single image,<br>
we are going to apply it for other images.

In [ ]:
# Sample images
sample_image = {}
for label in label_count.index:
    df_tmp = label_df[label_df.labels == label]
    image_file_names = df_tmp.sample(display_image_num).image.values
    images = [os.path.join(BASE_DIR, "train_images/", file_name) for file_name in image_file_names]
    sample_image[label] = images

In [ ]:
# Function to plot croped image 
def show_3x3_images(image_dict, min_thresh=100, max_thresh=200):
    display_image_num = 9
    for label, images in sample_image.items():
        # Show 3X3 images per each label
        plt.figure(figsize=(15, 8))
        for idx, image in enumerate(images):
            plt.subplot(math.ceil(display_image_num / 3), 3, idx + 1)
            img = plt.imread(image)
            get_and_plot_image_with_boundary(img, min_thresh, max_thresh)
        print("\n==================== Label:{} ===================".format(label))
        plt.tight_layout()
        plt.show()

We show the coroped images with different threshold.<br>
We could not find wich threshold might be the best, however we could croped leaf part from most of the images.<br>
I hope cropping images before applying to machine learning model would increase performance.

In [ ]:
show_3x3_images(sample_image, 5, 200)

In [ ]:
show_3x3_images(sample_image, 50, 200)

In [ ]:
show_3x3_images(sample_image, 100, 200)